# A notebook to proceed experiment on real dataset

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import autograd
from abc import ABCMeta, abstractmethod
import kgof.util as util
import kgof.data as data
import kgof.density as density
import kgof
import kgof.goftest as gof
import kgof.kernel as kernel
import kgof.glo as glo

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.ticker as ticker
import autograd.numpy as np
import scipy.stats as stats
from sklearn.neighbors import kde
from sklearn import mixture

In [ ]:
# font options
font = {
#     'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 32
}
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})


# matplotlib.use('cairo')
matplotlib.rc('text', usetex=True)
matplotlib.rcParams['text.usetex'] = True
plt.rc('font', **font)
plt.rc('lines', linewidth=2)
# matplotlib.rcParams['ps.useafm'] = True
# matplotlib.rcParams['pdf.use14corefonts'] = True

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42


In [ ]:
def filter_crimetype(data, type = None):
    if type is None:
        data = data
    else:
        data = data[data[:,0] == type]
    if len(data) == 1:
        print "No Crime Type found"
    else:
        loc = data[:,1:].astype(float)
        loc = np.nan_to_num(loc)
        loc = loc[loc[:,0] != 0]
        #Set City bound
        loc = loc[loc[:,0] >-89]
        loc = loc[loc[:,1] > 40]
        return loc

In [ ]:
def run_visual(loc_train, loc_test, size=10000, bin_size = 40):
    #Select Training data X, and Testing data T
    ds = data.DSResample(loc_train)
    dat, ind = ds.sample(size,seed=9,return_ind=True)
    tr, te, Itr = dat.split_tr_te(tr_proportion=0.5, return_tr_ind = True)
    X = tr.X
    indx = ind[Itr]
    T = te.X
    Total = dat.X
    ds_t = data.DSResample(loc_test)
    T2 = ds_t.sample(size, seed = 9).X

    X_H, X_x, X_y = np.histogram2d(X[:,0],X[:,1],bin_size)
#     X_H = X_H/np.sum(X_H)
    X_X, X_Y = np.meshgrid(X_x, X_y)

    T_H, T_x, T_y = np.histogram2d(T[:,0],T[:,1],bin_size)
#     T_H = T_H/np.sum(T_H)
    T_X, T_Y = np.meshgrid(T_x, T_y)

    T2_H, T2_x, T2_y = np.histogram2d(T2[:,0],T2[:,1],bin_size)
    rat = len(T2)/len(X)
    T2_H = T2_H/rat
#     T2_H = T2_H/np.sum(T2_H)
    T2_X, T2_Y = np.meshgrid(T2_x, T2_y)

    vmax = max(np.max(X_H),np.max(T_H),np.max(T2_H))
    norm_ = colors.Normalize(0,vmax)

    plt.figure(figsize = (5,5))
    hist = plt.pcolormesh(X_X,X_Y,X_H.T,norm=norm_)
    plt.axis([X_x.min(), X_x.max(), X_y.min(), X_y.max()])
    plt.title("Distribution of Training Set")
    plt.axis('off')
    
    plt.figure(figsize = (5,5))
    hist = plt.pcolormesh(T_X,T_Y,T_H.T,norm=norm_)
    plt.axis([T_x.min(), T_x.max(), T_y.min(), T_y.max()])
    plt.title("Distribution of Testing Set")
    plt.axis('off')
    
    plt.figure(figsize = (6.3,5))
    hist = plt.pcolormesh(T2_X,T2_Y,T2_H.T,norm=norm_)
    plt.axis([T2_x.min(), T2_x.max(), T2_y.min(), T2_y.max()])
    plt.title("Distribution of Second Year")
    plt.colorbar()
    plt.axis('off')
    
    return X,T,Total,T2,indx

In [ ]:
def fit_gmm(X, low = 140, high = 150, inter = 1):
    #Fit mixture of gaussian on X, optimising the number of clusters k using cv on BIC
    bic = []
    lowest_bic = np.infty
    n_components_range = range(low, high)
    cv_types = ['spherical']
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=inter*n_components,
                                          covariance_type=cv_type,random_state=3)
            gmm.fit(X)
            bic.append(gmm.bic(X))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmm = gmm
    return best_gmm

In [ ]:
def run_test(best_gmm, T, J = 1, opt=False, alpha = 0.05, Linear=True):
    # Setup the test on T
    seed = 4
    mean = best_gmm.means_
    variance = best_gmm.covariances_
    weight = best_gmm.weights_
    IsoGM = density.IsoGaussianMixture(means = mean, variances = variance,pmix=weight)
    ds = data.DSResample(T)
    dat = ds.sample(len(T))
    tr, te = dat.split_tr_te(tr_proportion=0.2, seed=seed+1)
    
    gwidth0 = util.meddistance(dat.X, subsample=1000)**2
    null_sim = gof.FSSDH0SimCovObs(n_simulate=1000, seed=10)
    if Linear is True:
        if opt is False:
            # random test locations
            V0 = util.fit_gaussian_draw(dat.X, J, seed=seed+1)
            k0 = kernel.KGauss(gwidth0)
            fssd = gof.FSSD(IsoGM, k0, V0, null_sim=null_sim, alpha=alpha)
            fssd_result = fssd.perform_test(dat)
            return fssd_result, V0, gwidth0
        else:
            opts = {
            'reg': 1e-1,
            'max_iter': 50, 
            'tol_fun':1e-7, 
            'disp':True,
            'gwidth_lb': 5e-4,
            'gwidth_ub': 1e-1}
            #Pick V0 randomly from training set to start with
            with util.NumpySeedContext(seed=seed+10):
                idx = np.random.choice(len(tr.X), size=J)
            V0 = tr.X[idx,:]
#             V0 = mean[:J,:]
            V_opt, gw_opt, opt_result = gof.GaussFSSD.optimize_locs_widths(IsoGM, tr, gwidth0, V0, **opts)
            k_opt = kernel.KGauss(gw_opt)
            fssd_opt = gof.FSSD(IsoGM, k_opt, V_opt, null_sim=null_sim, alpha=alpha)
            fssd_opt_result = fssd_opt.perform_test(dat, return_simulated_stats=False)
            return fssd_opt_result, V_opt, gw_opt
        
    else:
        sig2 = util.meddistance(X, subsample=1000)**2
        k = kernel.KGauss(sig2)
        bootstrapper = gof.bootstrapper_rademacher
        kstein = gof.KernelSteinTest(IsoGM, k, bootstrapper=bootstrapper, 
                                     alpha=alpha, n_simulate=1100, seed=seed+1)
        kstein_result = kstein.perform_test(dat, return_simulated_stats=False,
                                           return_ustat_gram=True)
        return kstein_result, bootstrapper, sig2

In [ ]:
def interp_plot(V_opt, T2, X, best_gmm, bin_size = 30, fitted_sample = True, scale=1.1,zoom=False):
    #Normalise count to density
    seed = 4
    np.random.seed(seed)
    X_H, X_x, X_y = np.histogram2d(X[:,0],X[:,1],bin_size)
    X_X, X_Y = np.meshgrid(X_x, X_y)
#     X_H = X_H/np.sum(X_H)
    
    if fitted_sample is True:
        sample_size = int(len(X)*scale)
        sample = best_gmm.sample(sample_size)
        samples = zoom_in(X_x.min(), X_x.max(), X_y.min(), X_y.max(),sample[0])
        if len(samples) < len(X):
            raise NameError('Samples not enough')
        else:
            dss = data.DSResample(samples)
            samples = dss.sample(len(X)).X
        Z,Z_x,Z_y = np.histogram2d(samples[:,0],samples[:,1],bin_size)
        Z_X, Z_Y = np.meshgrid(Z_x, Z_y)
    else:
        x = np.linspace(np.min(X[:,0]), np.max(X[:,0]),bin_size)
        y = np.linspace(np.min(X[:,1]), np.max(X[:,1]),bin_size)
        Z_x, Z_y = np.meshgrid(x, y)
        XX = np.array([Z_x.ravel(), Z_y.ravel()]).T
        Z = np.exp(best_gmm.score_samples(XX))
#         Z = Z.clip(min=1)-1
        Z = Z.reshape(Z_x.shape).T
#     Z = Z/np.sum(Z)

    T_H, T_x, T_y = np.histogram2d(T2[:,0],T2[:,1],bin_size)
    T_X, T_Y = np.meshgrid(T_x, T_y)
    rat = len(T2)/float(len(X))
    T_H = T_H/rat
#     T_H = T_H/np.sum(T_H)
    
    vmax = max(np.max(X_H),np.max(T_H),np.max(Z))
    norm_ = colors.Normalize(0,vmax)
    
    xmin = max(X_x.min(),T_x.min(),Z_x.min())
    xmax = min(X_x.max(),T_x.max(),Z_x.max())
    ymin = max(X_y.min(),T_y.min(),Z_y.min())
    ymax = min(X_y.max(),T_y.max(),Z_y.max())
    xs = np.linspace(xmin,xmax,4)
    ys = np.linspace(ymin,ymax,4)
    font_size = 20
    if zoom is True:
        z = '_Zoomed_In'
    else:
        z = ''
    
    #Plot density histogram
    plt.figure(figsize = (5,5))
#     hist = plt.hist2d(X[:,0],X[:,1],bin_size,norm=norm_)
    plt.pcolormesh(X_X,X_Y,X_H.T,norm=norm_)
    plt.axis([xmin, xmax, ymin, ymax])
    plt.xticks(xs)
    plt.yticks(ys)
    ax = plt.gca()
    xax = ax.get_xaxis()
    yax = ax.get_yaxis()
    xax.set_ticklabels(['%.2f'%x for x in xs],fontsize=font_size)
    yax.set_ticklabels(['%.2f'%x for x in ys],fontsize=font_size)
    plot_name = "Distribution_of_Training_Set" + z
#     plt.title(plot_name)
    plt.savefig(plot_name+'.pdf',bbox='tight')   
    
    plt.figure(figsize = (5,5))
#     CS = plt.contourf(Z_x, Z_y, Z)
    plt.pcolormesh(Z_x, Z_y, Z.T, norm=norm_)
    plt.axis([xmin, xmax, ymin, ymax])
    plt.xticks(xs)
    plt.yticks(ys)    
    ax = plt.gca()
    xax = ax.get_xaxis()
    yax = ax.get_yaxis()
    xax.set_ticklabels(['%.2f'%x for x in xs],fontsize=font_size)
    yax.set_ticklabels(['%.2f'%x for x in ys],fontsize=font_size)
    plot_name = "Fitted_Gaussian_Mixture" + z
#     plt.title(plot_name)
    plt.savefig(plot_name+'.pdf',bbox='tight')
    
    plt.figure(figsize = (6.3,5))
#     hist = plt.hist2d(T2[:,0],T2[:,1],bin_size,norm=norm_)
    plt.pcolormesh(T_X,T_Y,T_H.T,norm=norm_)
    plt.axis([xmin, xmax, ymin, ymax])
    plt.xticks(xs)
    plt.yticks(ys)
    ax = plt.gca()
    xax = ax.get_xaxis()
    yax = ax.get_yaxis()
    xax.set_ticklabels(['%.2f'%x for x in xs],fontsize=font_size)
    yax.set_ticklabels(['%.2f'%x for x in ys],fontsize=font_size)
    plt.colorbar()
    plt.plot(V_opt[:,0], V_opt[:,1], 'w*', markersize=25, alpha=0.6)
    plot_name = "Interpretable_Location" + z
#     plt.title(plot_name)
    plt.savefig(plot_name+'.pdf',bbox='tight')

In [ ]:
def power_check(loc, bmm, indx = None, itr=10, alpha = 0.05, opt=False, Linear = True):
    rej = 0
    rej_quad = 0
    fssd_res = []
    if indx is None:
        test = loc
    else:
        mask = np.ones(len(loc), np.bool)
        mask[indx] = 0
        test = loc[mask]
    with util.NumpySeedContext(seed=48):
        for i in range(itr):
            ds = data.DSResample(test)
            dat = ds.sample(len(indx),np.random.randint(len(test)))
            fssd = run_test(best_gmm, dat.X, opt=opt, alpha = alpha, Linear = Linear)
            fssd_res.append(fssd)
            rej += fssd[0]['h0_rejected']
    return rej, fssd_res

In [ ]:
def zoom_in(x_low,x_high,y_low,y_high,loc):
    loc = loc[loc[:,0] > x_low]
    loc = loc[loc[:,1] > y_low]
    loc = loc[loc[:,0] < x_high]
    loc = loc[loc[:,1] < y_high]
    return loc

## Chicago Crime Dataset is used

In [ ]:
#Extract data from pre-processed .npz file
dd_new = np.load(glo.data_file('chicago_crime_loc_with_type2016.npz'))['data']
dd_old = np.load(glo.data_file('chicago_crime_loc_with_type2014.npz'))['data']
old_year = 2014
new_year = 2016

In [ ]:
from collections import Counter
def crime_count(data, n=10000):
    z = data[:,0]
    count = Counter(z)
    interest = []
    for i in enumerate(count):
        if count[i[1]] > n:
            interest.append([i[1],count[i[1]]])
    return interest
interestnew = crime_count(dd_new)
interestold = crime_count(dd_old)
print 'old_year:', interestold
print 'new_year:',interestnew

In [ ]:
#Choose the type of Crime category if necessary and proceed
c_type = 'BURGLARY'
size = 10000
loc_old = filter_crimetype(dd_old, c_type)
loc_new = filter_crimetype(dd_new, c_type)

#Extract Training, Testing dataset and necessary index
X,T,Total,T2,indx = run_visual(loc_old,loc_new,size=size, bin_size=40)

In [ ]:
#Build the Gaussian Miture model
best_gmm = fit_gmm(X,low = 38, high = 39, inter=1)
n_comp = best_gmm.n_components
best_gmm

In [ ]:
#Test this year
fssd_this_year = run_test(best_gmm,T,opt=False)
fssd_this_year

In [ ]:
#Check power on the FSSD for fitted model
# iteration = 20
# rej, fssd_res= power_check(loc_old, bmm = best_gmm, indx = indx, itr = iteration, \
#                            alpha = 0.05, opt=True, Linear = True)
# rate = rej/float(iteration)
# print rate

In [ ]:
#Check power on original quadratic time KSTein Test
# iteration = 20
# rej_quad, fssd_res_quad= power_check(loc_old, bmm = best_gmm, indx = indx, itr = iteration, \
#                            alpha = 0.05, opt=True, Linear = False)
# rate_quad = rej_quad/float(iteration)
# print rate_quad

In [ ]:
#Fit the model 
best_gmm = fit_gmm(Total,low = n_comp, high = n_comp+1, inter=1)
best_gmm

In [ ]:
#Test on next year data without optimised location
fssd_next_year = run_test(best_gmm, T2)
fssd_next_year

In [ ]:
#Test on next year data with optimised location
n_points = 1
fssd_next_year_opt, V_opt, gw_opt = run_test(best_gmm, T2, J=n_points, opt=True)
fssd_next_year_opt

In [ ]:
V_opt

In [ ]:
gw_opt

## Plot the intensity, samples from fitted model and test location on testing set

In [ ]:
interp_plot(V_opt,T2, Total, best_gmm = best_gmm, bin_size = 40, fitted_sample=True)

Possible V_opt location:

[[-87.77217121,  42.01658179],

[-87.59425177,  41.78447888]]

## Zoom in the region around optimised testing location

In [ ]:
box_size = 0.07
V_loc = V_opt[0,:]

x_low = V_loc[0] - box_size
x_high = V_loc[0] + box_size
y_low = V_loc[1] - box_size
y_high = V_loc[1] + box_size
Total_zoom = zoom_in(x_low,x_high,y_low,y_high,Total)
T2_zoom = zoom_in(x_low,x_high,y_low,y_high,T2)

scale = float(len(Total))/len(Total_zoom)*2
interp_plot(V_opt,T2_zoom, Total_zoom, best_gmm = best_gmm, bin_size = 40, \
            fitted_sample=False, scale = scale, zoom=True)

In [ ]:
# Save results in .npz file
# np.savez('chicago_'+str(c_type)+'comparison_with'+str(n_comp)+'GMM'+str(n_points)+'locations.npz',\
#          crime_type = c_type, old_year = old_year, new_year = new_year, V_opt = V_opt, n_comp = n_comp, gw_opt = gw_opt,\
#          fssd_this_year = fssd_this_year, fssd_next_year = fssd_next_year, fssd_next_year_opt = fssd_next_year_opt)         